<a href="https://colab.research.google.com/github/pritiyadav888/AI-projects/blob/main/populationVS_housingvacancy_rateV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Load datasets
df_vaca_1 = pd.read_csv('https://raw.githubusercontent.com/pritiyadav888/Housing-datasets/main/3410012801_databaseLoadingData.csv')
df_vaca_2 = pd.read_csv('https://raw.githubusercontent.com/pritiyadav888/Housing-datasets/main/3410012901_databaseLoadingData.csv')
df_popul_1 = pd.read_csv('https://raw.githubusercontent.com/pritiyadav888/Housing-datasets/main/Population%20estimates_1.csv')
df_popul_2 = pd.read_csv('https://raw.githubusercontent.com/pritiyadav888/Housing-datasets/main/Population%20estimates_2.csv')

# Merge datasets
merged_vacan = pd.concat([df_vaca_1, df_vaca_2], join='inner')
merged_popul = pd.concat([df_popul_1, df_popul_2], join='inner')

# Rename columns for better understanding
def rename_columns(df):
    return df.rename(columns={'VALUE': 'value', 'GEO': 'geo', 'REF_DATE': 'year'})

merged_vacan = rename_columns(merged_vacan)
merged_popul = rename_columns(merged_popul)

# Drop unnecessary columns
def drop_columns(df):
    columns_to_drop = ['STATUS', 'SYMBOL', 'TERMINATED', 'DECIMALS']
    return df.drop(columns=columns_to_drop, errors='ignore')

merged_vacan = drop_columns(merged_vacan)
merged_popul = drop_columns(merged_popul)

# Handle missing values (fillna with 0 - change this to another method if you prefer)
merged_vacan['value'].fillna(0, inplace=True)

# Create a function to extract the province name from the 'geo' column
def extract_province(geo):
    provinces = ['Newfoundland and Labrador', 'Prince Edward Island', 'Nova Scotia', 'New Brunswick', 'Quebec', 'Ontario', 'Manitoba', 'Saskatchewan', 'Alberta', 'British Columbia', 'Yukon', 'Northwest Territories', 'Nunavut']
    for province in provinces:
        if province in geo:
            return province
    return None

# Apply the extract_province function to each dataset
merged_popul['province'] = merged_popul['geo'].apply(extract_province)
merged_vacan['province'] = merged_vacan['geo'].apply(extract_province)

# Group the data by 'province' and 'year' and calculate the mean (or sum) for each group
grouped_popul = merged_popul.groupby(['province', 'year']).sum().reset_index()
grouped_vacan = merged_vacan.groupby(['province', 'year']).mean().reset_index()

# Merge the two datasets on the 'province' and 'year' columns
merged_data = pd.merge(grouped_vacan, grouped_popul, on=['province', 'year'])

# Calculate percentage changes and add as new columns
merged_data['population_change_pct'] = merged_data.groupby('province')['value_y'].pct_change() * 100
merged_data['vacancy_rate_change_pct'] = merged_data.groupby('province')['value_x'].pct_change() * 100

<ipython-input-8-3093f4f30236>:48: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

<ipython-input-8-3093f4f30236>:49: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [9]:
# Filter the data based on a specific year (e.g., 2020)
before_covid = merged_data[merged_data['year'] < 2020]
after_covid = merged_data[merged_data['year'] >= 2020]

# Create a D3 visualization using Plotly
# Create a D3 visualization using Plotly and save as HTML
fig1 = px.line(merged_data, x='year', y='population_change_pct', color='province')
fig1.update_layout(title='Population Change Over Years by Province', xaxis_title='Year', yaxis_title='Population Change (%)')
fig1.write_html("population_change.html")

fig2 = px.line(merged_data, x='year', y='vacancy_rate_change_pct', color='province')
fig2.update_layout(title='Vacancy Rate Change Over Years by Province', xaxis_title='Year', yaxis_title='Vacancy Rate Change (%)')
fig2.write_html("vacancy_rate_change.html")

fig3 = px.line(before_covid, x='population_change_pct', y='vacancy_rate_change_pct', color='province',
                 title='Population Change vs. Vacancy Rate Change by Province Before COVID-19')
fig3.update_layout(xaxis_title='Population Change (%)', yaxis_title='Vacancy Rate Change (%)')
fig3.write_html("scatter_before_covid.html")

fig4 = px.line(after_covid, x='population_change_pct', y='vacancy_rate_change_pct', color='province',
                 title='Population Change vs. Vacancy Rate Change by Province After COVID-19')
fig4.update_layout(xaxis_title='Population Change (%)', yaxis_title='Vacancy Rate Change (%)')
fig4.write_html("scatter_after_covid.html")

table_data_before = before_covid.groupby('province')[['population_change_pct', 'vacancy_rate_change_pct']].mean()
table_data_after = after_covid.groupby('province')[['population_change_pct', 'vacancy_rate_change_pct']].mean()

table_data = pd.concat([table_data_before, table_data_after], axis=1, keys=['Before COVID', 'After COVID'])
table_data.index.name = 'Province'
table_data.reset_index(inplace=True)

# Save the numeric table as HTML
table_data_html = table_data.to_html(index=False)
with open('numeric_table.html', 'w') as f:
    f.write(table_data_html)

In [12]:
# Read the HTML files
with open('population_change.html') as f:
    population_change_html = f.read()
    
with open('vacancy_rate_change.html') as f:
    vacancy_rate_change_html = f.read()
    
with open('scatter_before_covid.html') as f:
    scatter_before_covid_html = f.read()
    
with open('scatter_after_covid.html') as f:
    scatter_after_covid_html = f.read()
    
with open('numeric_table.html') as f:
    numeric_table_html = f.read()

# Add margin and padding to the numeric table
numeric_table_html = numeric_table_html.replace('<table>', '<table style="margin: 20px; padding: 10px;">')

# Merge the HTML files
housing_vacancy_rate = f"""
<html>
<head>
    <title>Housing Vacancy Rate vs Population</title>
</head>
<body>
    <p>"Note: As the analyst, I have presented the findings and conclusions in this HTML file based on the data available at the time of analysis and the methods used to process and visualize the data. While these insights may be helpful for certain purposes, they should be considered as exploratory and not definitive. It is recommended that you conduct your own research and analysis to validate and supplement the information presented here. Please be aware that the level of detail and technicality of the analysis may vary depending on the experience and expertise of the analyst. As such, exercise caution and do not blindly rely on the information presented without conducting your own due diligence."</p>
    <p></p>
    {population_change_html}
    {vacancy_rate_change_html}
    {scatter_before_covid_html}
    {scatter_after_covid_html}
    {numeric_table_html}
</body>
</html>
"""

# Save the merged HTML file
with open('housing_vacancy_rate.html', 'w') as f:
    f.write(housing_vacancy_rate)